# Librerias

In [1]:
#Importemos las librerías Necesarias:
import scipy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random, time

In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from keras.layers import *
from keras.models import *

***

# Main Code

Dado que En intentos anteriores se ha recibido un error, haremos lo siguiente:

In [3]:
# !nvcc --version 
# !nvidia-smi

# Veamos los dispositivos Fisicos de tipo GPU:
ph_dev = tf.config.experimental.list_physical_devices('GPU')
print(ph_dev)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
(X_train, y_train), (X_test, y_test) = tfds.as_numpy(
    tfds.load(
            name='rock_paper_scissors',
            split=['train', 'test'],
            batch_size=-1,
            as_supervised=True
        )
    )

Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


In [5]:
# Veamos a que nos enfrentamos:
X_train[0].shape, y_train.shape

((300, 300, 3), (2520,))

De aquí sabemos que las imágenes tienen 300x300 pixeles, ademas como son imágenes en RGB cada una de ellas tiene una profundidad de 3 capas, de modo que cada una de ellas mostrará en el rango 0, 255 la intensidad del color en el pixel analizado.

In [6]:
# Reescalemos primero las imagenes a fin de ocupar menos memoria:
train, test = list(), list()
size = 120
for image in X_train:
    train.append(tf.image.resize(image, [size,size]))
for image in X_test:
    test.append(tf.image.resize(image, [size,size]))

X_test = np.array(train)
X_train = np.array(train)
y_train = keras.utils.to_categorical(y_train, 3)
y_test = keras.utils.to_categorical(y_test, 3)
    
X_test.shape, y_test.shape

((2520, 120, 120, 3), (372, 3))

In [7]:
# escalemos los datos al rango 0, 1:
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [8]:
# Creemos el modelo:
model = Sequential()
model.add(Conv2D(
    filters=32,
    kernel_size=(3,3),
    activation='relu',
    input_shape=X_train[0].shape))
model.add(MaxPooling2D((3,3)))
model.add(Conv2D(
    filters=64,
    kernel_size=(3,3),
    activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 118, 118, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 39, 39, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 37, 37, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 18, 18, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 20736)             0         
                                                                 
 dense (Dense)               (None, 64)                1

In [9]:
model.fit(
    X_train,
    y_train, 
    epochs=20,
    batch_size=128,
    verbose=1,
    validation_data=(X_test, y_test)
)

Epoch 1/20


***

## Notas:

> Keras.Conv2D:
>   * filters: int(), Dimension de la salida de la convolución
>   * kernel_size: int / tuple(int, int) / list(int, int), matriz de convolución (n_px * n_px)

## Bibliografia:

   * Stack Overflow: https://stackoverflow.com
   * Elemento bibliografico 2

### Autor

   - **Name:** Elidas
   - **Email:** pyro.elidas@gmail.com
   - **Python version:** 3.9.1
   - **Date:** ${DATE}